# CMM262: RNA-sequencing, Day 2<br>Part 2 - (RNA-seq Differential Expression Analysis)

**Authors:** Michelle Franc Ragsac (mragsac@eng.ucsd.edu) and Eric Kofman (ekofman@eng.ucsd.edu)

> *Based on the DESeq R Markdown notebook from CMM262 taught in Winter 2020* 

Now that we've gone through the beginning of the RNA-sequencing analysis pipeline to transform our FASTQ sequencing data to, eventually, a counts matrix to show the number of reads we have for each gene in our experiment, we can use the `DESeq2` package in the R programming language to determine the *differentially expressed* genes present in our experiment. 

---

### Table of Contents

1. Performing Differential Expression Analyses with the `DESeq2` Library 
2. Defining Experimental Parameters for `DESeq2` and Constructing the `DESeqDataSet` Object
3. Performing Data Quality Control with Built-In `DESeq2` Methods
4. Perform Differential Expression Analysis on our Dataset
5. Generating MA-Plots to Determine the Differences between Samples

---

## Performing Differential Expression Analyses with the `DESeq2` Library 

> Love, Michael I., Wolfgang Huber, and Simon Anders. "Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2." *Genome biology* 15.12 (2014): 550. https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8

One of the most basic tasks in RNA-sequencing analysis is the detection of differentially expressed genes. Often, the count data that comes out of a command-line RNA-sequencing processing pipeline comes as a table that reports the number of sequence fragments (e.g., reads) that have been assigned to each gene of interest. 

However, there are some challenges that need to be considered when trying to assess quantitative differences between experiments: (i) biological count data is often *non-normal*, (ii) variance on the mean of count data can vary between samples, and (iii) there are often a small number of sample per condition to start off with! Because methods that treat each gene separately lack power due to the uncertainty of within-group variance estimates, many methods pool information across genes by exploiting assumptions about the similarity of the variances (or *dispersion*) of different genes measured in the same experiment. 

<div class="alert alert-block alert-info">
    <p>If you're interested in why it's important to normalize and model count data accurately, check out the following Jupyter Notebook that was presented at the 2020 BISB Bootcamp's <i>Introduction to Machine Learning</i> module taught by Cameron Martino: <a href="https://github.com/mragsac/BISB-Bootcamp-2020/blob/master/day4/module7_machine-learning-101/2.0-count-data-intro.ipynb">https://github.com/mragsac/BISB-Bootcamp-2020/blob/master/day4/module7_machine-learning-101/2.0-count-data-intro.ipynb</a>!</p>
</div>

Within this notebook, we'll be focusing on the popular software known as `DESeq2`. To tackle the problem of comparing counts data between different biological samples, `DESeq2` first models the raw counts that are observed using *normalization factors* (otherwise known as *size factors*) to account for differences in library depth. Then, it estimates the variability of all of the genes present (or *gene-wise dispersions*) and shrinks the estimates to generate more accurate representations of dispersion of model the raw counts. Finally, using a generalized linear model (GLM), `DESeq2` fits the data to a *negative binomial model* and performs hypothesis testing using the *Wald test* or *Likelihood Ratio Test* to determine differentally expressed genes.

This notebook will focus on giving you the basic commands to use `DESeq2` on your own data! We'll be using the human airway smooth muscle transcriptome dataset that we used in Day 3 of our statistics module for this demonstration: https://github.com/biom262/cmm262-2021/blob/main/module-2-statistics/Day%203%20-%20Gene%20Expression%20Analysis.ipynb.

<div class="alert alert-block alert-info">
    <p>Because <code>DESeq2</code> is a popular package for differential expression analysis in R, there are a lot of resources available for learning how to use the tool. Here are some of our favorites that we would recommend:</p>
    <ul>
        <li><b>Official DESeq2 Documentation</b>:<a href="https://bioconductor.org/packages/release/bioc/manuals/DESeq2/man/DESeq2.pdf">https://bioconductor.org/packages/release/bioc/manuals/DESeq2/man/DESeq2.pdf</a></li>
        <li><b>Official DESeq2 Vignette</b>:<a href="http://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html">http://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html</a></li>
        <li><b>Beginner's Guide to Using the DESeq2 Package by Love <i>et al.</i></b>:<a href="http://www.bioconductor.org/packages/2.14/bioc/vignettes/DESeq2/inst/doc/beginner.pdf">http://www.bioconductor.org/packages/2.14/bioc/vignettes/DESeq2/inst/doc/beginner.pdf</a></li>
    </ul>
</div>

### Import the Packages We'll be Using in this Portion of the Notebook

In [ ]:
# First, import/load in the DESeq2 library into our notebook
library(DESeq2)

# Next, import/load in the ggplot2 and RColorBrewer packages for result exploration
library(ggplot2)
library(RColorBrewer)

### Load in the RNA-Sequencing Dataset using the `read.csv()` Method on our Counts Matrix and Labels File

<div class="alert alert-block alert-info">
    <p>The values in the count matrix should be <b>un-normalized counts</b> or estimated counts of sequencing reads (for single-end RNA-sequencing data) or fragments (for paired-end RNA-sequencing data). It is important to provide count matrices for <code>DESeq2</code> for the software's statistical model to hold, as only the count values allow assessing the measurement precision correctly.</p>
    <p><code>DESeq2</code> internally corrects for library size, so transformed or normalized values such as counts scaled by library size should <b>not</b> be used as input!</p>
</div>

In [ ]:
# Because DESeq2 works with raw count matrices and uses the row names as identifiers 
# for our genes, we'll import the data accordingly with the read.csv() method!
counts <- read.csv('data/asm_dex_counts.txt', 
                   sep = "\t",    # specify that our data is tab-delimited 
                   row.names = 1, # designate the row with gene names present
                   header = TRUE) # state that we have a header already present

# After importing in the data, let's preview the contents with the head() method
head(counts)

In [ ]:
# We also need our condition identifiers so DESeq2 knows what groups to compare against each other 
col.data <- read.csv('data/asm_dex_labels.txt', sep = '\t', header = TRUE, row.names = 1)
head(col.data)

---

## Defining Experimental Parameters for `DESeq2` and Constructing the `DESeqDataSet` Object

The object class used by the `DESeq2` package to store the read counts and intermediate estimated quantities during statistical analysis is known as the `DESeqDataSet`, which is usually represented in code as the `dds` variable. 

One of the most important components is that the `DESeqDataSet` must have an associated *design formula*, provided by the `design` parameter. The design formula is used to estimate the dispersions and to estimate the $log_2$ fold changes of the model. The design formula expresses the variables that will be used in the modeling and should be a tilde (`~`) followed by the variables (if there are multiple variables, then they are connected by plus (`+`) signs). While the design can be changed later, it is recommended that all of the differential analysis steps be repeated if it is modified. 

<div class="alert alert-block alert-warning">
    <p>While there are multiple ways of constructing the <code>DESeqDataSet</code> depending on what the upstream steps were, we will be using the <code>DESeqDataSetFromMatrix()</code> method to provide count matrix input from our RNA-sequencing data processing pipeline that we performed earlier!</p>
</div>

<div class="alert alert-block alert-info">
    <p>If you have additional feature data, it can be added to the <code>DESeqDataSet</code> by adding to the metadata columns of a newly constructed object (e.g., you can add technical replicate or sample batch information in addition to the different sample conditions being tested).</p>
</div>

In [ ]:
# Generate the DESeq2DataSet object using our counts matrix values and labels 
dds <- DESeqDataSetFromMatrix(countData = counts,   # specify the counts matrix to use 
                              colData = col.data,   # specify our sample groupings 
                              design = ~ condition) # state we would like to model the condition of our groupings

# Now that we've generate the DESeq2DataSet object, let's preview the contents of the object!
dds 

---

## Performing Data Quality Control with Built-In `DESeq2` Methods

### Applying a Regularized Log-Transformation with the `rlog()` Method

Within RNA-sequencing data, we see the phenomenon that within a dataset, there are a few genes that have very strong expression and these genes are often outnumbered by the genes with low or moderate expression. This makes it difficult to reduce the dimensionality of RNA-sequencing datasets with common statistical methods, such as principal component analysis (PCA), without performing an additional transformation step beforehand. A common transformation step is to take the logarithm of normalized count values plus a small pseudocount; however, due to the strong Poisson noise inherent to small count values, this leads genes with low counts to dominate the results! 

As a solution, `DESeq2` uses a method called the *regularized-logarithm transformation*, found in the `rlog()` function. For genes with high counts, the `rlog()` function does not differ much from a regular $log_2$ transformation. However, for genes with lower counts, the values are *shrunken* towards the genes' averages across all samples! 

<div class="alert alert-block alert-info">
    <p>The regularized-logarithm transformation <b>should not</b> be used for differential testing. Again, is recommended that you use raw counts for the differential testing methods offered by <code>DESeq2</code>.</p>
</div>

In [ ]:
rld <- rlog(dds)

### Visualizing Sample Separation with Principal Component Analysis (PCA) via the `plotPCA()` Method

A useful step within RNA-sequencing analyses is to evaluate the overall similarity between samples: which samples are different versus similar to each other? One way to visualize these differences is using principal component analysis, or PCA. Within this method, the data points are projected into two-dimensional space such that they're able to spread out according to their variances across different axes. The `plotPCA()` function provides an easy way to visualize these differences!

In [ ]:
# Calculate the PCA dimensionality reduction of our samples with the plotPCA() method
data <- plotPCA(rld, intgroup = 'condition', returnData = TRUE)

# Calculate the variance that is captured by each principal component
percent.var <- round(100 * attr(data, 'percentVar'))

# Generate the PCA plot with ggplot
ggplot(data, aes(x = PC1, y = PC2, color = condition)) +
    geom_point(size = 5) + 
    xlab(paste("PC1: ", percent.var[1], "%variance")) +
    ylab(paste("PC2: ", percent.var[2], "%variance"))

### Visualizing Sample Similarities with a Heatmap using the `heatmap()` Method

Another way to visualize the differences between samples is to calculate the distance between them using the *Euclidean distance* offered by the built-in R function, `dist()`. However, to avoid that the distance measure is dominated by a few highly-variable genes and have roughly equal contribution from all genes, we can use this `dist()` function on the `rlog`-transformed data that we generated earlier.

In [ ]:
# Calculate the distances of our rlog-transformed data; however, we need to use the t() function 
# to transpose the data matrix as dist() calculates distances between rows, and our samples are contained in the columns
sample.distances <- dist(t(assay(rld)))

# Transform the values to a matrix object and label the rows and columns appropriately by sample names
sample.distances.matrix <- as.matrix(sample.distances)
rownames(sample.distances.matrix) <- paste(rld$condition)
colnames(sample.distances.matrix) <- paste(rld$condition)

# Preview the distances matrix
head(sample.distances.matrix)

In [ ]:
# Determine the color scheme that we want to use then plot the heatmap using that color scheme
colors <- colorRampPalette(rev(brewer.pal(9, 'Blues')))(255)
heatmap(sample.distances.matrix, col = colors)

---

## Perform Differential Expression Analysis on our Dataset

Luckily, despite all of the math behind it, it's fairly easy to run the functions for differential expression analyses using `DESeq2`! The method that allows us to perform the differential expression analysis on the conditons, or `design`, that we specified earlier is the `DESeq()` method. 

Afterwards, the `results()` method can be used to generate the results table. This extracts a table with $log_2$ fold changes, p-values, and adjusted p-values. 

In [ ]:
# Perform the differential analysis step
dds.result <- DESeq(dds)

In [ ]:
# Generate the results table and preview the results
result <- results(dds.result)
head(as.data.frame(result))

In [ ]:
# The summary() method allows us to generate some basic tallies about our data
summary(result)

In [ ]:
# We can also re-order the results table according to the adjusted p-values (smallest to largest),
# so let's do that, then store the sorted table under the same variable 
result <- result[order(result$padj),]
head(as.data.frame(result))

### Examining the Counts of Reads for a Single Gene Across Groups

It can be useful to examine the counts of reads for a single gene across our different design parameters. A simple function to do this is the `plotCounts()` method, which normalizes counts by sequencing depth and adds a small pseudocount to allow for log-scale plotting. The counts are grouped by the variables in the `intgroup` attribute, where more than one variable can be specified. 

In the examples below, we've seleted some random genes to visualize!

In [ ]:
# Organize the plots in a 3x2 grid
par(mfrow=c(3,2))

# Plot some genes that we've randomly selected
plotCounts(dds, gene="ENSG00000152583") 
plotCounts(dds, gene="ENSG00000179094")
plotCounts(dds, gene="ENSG00000116584") 
plotCounts(dds, gene="ENSG00000189221")
plotCounts(dds, gene="ENSG00000120129")
plotCounts(dds, gene="ENSG00000148175")

---

## Generating MA-Plots to Determine the Differences between Samples

MA plots are a way to visualize gene expression datasets in two dimensions, and are especially useful for experiments with two-group comparisons. Within a MA plot, each gene is reprsented as a single dot. The x-axis is the average expression across all samples, and the y-axis represents the $log_2$ fold change between the two conditions being observed (e.g., control versus dex-treated samples). Genes with an adjusted p-value below some threshold are then shown in red. 

Generally, **genes with similar expression values in both conditions will cluster around `M=0`, whereas points away from `M=0` indicate genes that have significant expression.** Genes above `M=0` are considered to be *upregulated*, whereas genes below `M=0` are considered to be *downregulated*. It is also important to note that `DESeq2` has a specific way for how it considered log fold changes: when count values are too low to allow for an accurate estimation of the log fold change, the value is "shrunken" towards `0` to avoid overestimation of its ranking across all genes. 

<div class="alert alert-block alert-info">
    <p>MA plots demonstrate that only genes with a *large* average normalized count contain sufficient information to yield a significant call. That is, for weakly expressed genes, we have no chance at seeing differential expression because the low read counts suffer from so much noise that any biological effect is drowned out by uncertainties in read counting.</p>
</div>

In [ ]:
plotMA(result, main = "DESeq2 MA", ylim = c(-2,2))

In [ ]:
result <- results(dds.result, alpha = 0.05)
result.dataframe <- as.data.frame(result)

plotMA(result, main = "DESeq2 MA, alpha=0.05", ylim = c(-2,2))

---

## Visualizing Significant Results with Volcano Plots

Finally, the most common method of visualization to visualize the results of a differential expression analysis is through *volcano plots*. Volcano plots represent a useful way to visualize the results of differential expression analyses. They're a type of scatterplot that shows the statistical significance versus magnitude of change, that is, p-value versus log fold change! They allow for the quick identification of genes with large fold changes that are *also* statistically significant, as these points might be the most biologically significant genes to look at for further study. Within volcano plots, the most upregulated genes are towards the *right* and the most downregulated genes are towards the *left*, while the most statistically significant genes are towards the top. 

In [ ]:
# First, compute significances, as well as determine the different thresholds for significance 
result.dataframe$neg.log10.padj <- -log10(result.dataframe$padj)
result.dataframe$is.sig <- result.dataframe$padj < 0.05
result.dataframe$is.sig.big.fc <- result.dataframe$is.sig & (result.dataframe$log2FoldChange > 2 | result.dataframe$log2FoldChange < -2)

In [ ]:
# Generate the volcano plot
ggplot(result.dataframe, aes(x = log2FoldChange, y = neg.log10.padj, color = is.sig.big.fc)) +
    geom_point(size = 1) +
    scale_color_manual(values = c("black", "red")) +
    xlab("Log2FC Normalized Counts") +
    ylab("-Log10 Adjusted p-Value") 

<div class="alert alert-block alert-info">
    <p>Within these plots, the y-axis uses the negative p-value scale so that the smallest p-values (the most significant) are at the top of the plot.</p>
</div>

#### Assessing the Number of Significant Genes after Evaluating the Volcano Plot

In [ ]:
# Let's first get the dimensions that we have originally...
# Using dim(), we can figure out how many genes we have to start within our dataset
dim(result.dataframe)

In [ ]:
# Afterwards, let's filter our dataset according to the genes that have a big fold change!
# (We're also going to store these points under a new variable called sig.results.dataframe)
sig.results.dataframe <- result.dataframe[result.dataframe$is.sig.big.fc,]

dim(sig.results.dataframe) # show the dimensions

In [ ]:
# Finally, from the dataframe with big fold changes, let's figure out the number that have a non-null adjusted p-value
sig.results.dataframe.filt <- sig.results.dataframe[-which(is.na(sig.results.dataframe$padj)),]

dim(sig.results.dataframe.filt) # show the dimensions

In total, there seem to be `240` genes that we were able to detect with a high magnitude of change (log fold change) and also show statistical significance (p-value)!

---

## Using the `biomaRt` Library to Convert ENSEMBL Gene Identifiers to HSNC Gene Symbols 

### Import the Packages We'll be Using in this Portion of the Notebook

In [ ]:
library("biomaRt")

### Create the `ENSEMBL` BioMart Object with the `useDataset()` and `useMart()` Methods

In [ ]:
ensembl <- useDataset("hsapiens_gene_ensembl", useMart("ensembl", host="uswest.ensembl.org"))
ensembl

### Extract and Clean `ENSEMBL` Identifiers from our `DESeq2` Results with the `gsub()` Method

In [ ]:
ensembl.ids <- rownames(sig.results.dataframe)

### Fetch Conversion Between `ENSEMBL` Identifiers and HGNC Symbols with the `getBM()` Method

In [ ]:
# Before we fetch the gene list, let's enter these commands to 
# combat connection problems we might encounter when accessing biomart... 
# (From: https://www.bioconductor.org/packages/devel/bioc/vignettes/biomaRt/inst/doc/accessing_ensembl.html)
httr::set_config(httr::config(ssl_verifypeer = FALSE))
httr::set_config(httr::config(ssl_cipher_list = "DEFAULT@SECLEVEL=1"))

# Fetch the translation for our ENSEMBL IDs from BioMart to convert them to HGNC Symbols!
gene.list <- getBM(filters = 'ensembl_gene_id', 
                   attributes = c('ensembl_gene_id', 'hgnc_symbol'), # specifies columns we want from biomart
                   values = ensembl.ids, # provide the query for our search - which ENSEMBL IDs to look up
                   mart = ensembl)       # provide the biomart object we would like to use for our search

### Translate the `ENSEMBL` Identifiers in our `DESeq2` Database

In [ ]:
rownames(sig.results.dataframe) <- ensembl.ids
filtered.sig.results.dataframe <- sig.results.dataframe[gene.list$ensembl_gene_id,]

rownames(filtered.sig.results.dataframe) <- make.names(gene.list$hgnc_symbol, unique = TRUE)
head(filtered.sig.results.dataframe)